In [11]:
import numpy as np
from bs4 import BeautifulSoup
from urllib2 import urlopen

pseudocode

go to company list
scrape all companies
for each company:
    record info (unique id, # of pioneers, departure date, arrival date)
    go to company page
    record more info (type, category, direction, captain)
    then go to roster tab
    scrape each person
    for each person:
        record info (age, birthdate, deathdate)
        go to person page
        scrape FamilySearch ID (if it's there)
        scrape gender

In [12]:
page = urlopen('https://history.lds.org/overlandtravel/companies')
soup = BeautifulSoup(page)

In [15]:
print soup.tbody.prettify()

<tbody>
 <tr>
  <td class="sort-by-name" data-value="Alexander">
   <a href="/overlandtravel/companies/402/horace-m-alexander-company">
    Horace M. Alexander Company (1847)
   </a>
  </td>
  <td class="sort-by-number-of-pioneers" data-value="6">
   6
  </td>
  <td class="sort-by-departure-date" data-value="1847-07-24">
   24 July 1847
  </td>
  <td class="sort-by-arrival-date" data-value="1847-10-16">
   16 October 1847
  </td>
 </tr>
 <tr>
  <td class="sort-by-name" data-value="Allred">
   <a href="/overlandtravel/companies/78747837894925591810/levi-allred-company">
    Levi Allred Company (1851)
   </a>
  </td>
  <td class="sort-by-number-of-pioneers" data-value="9">
   9
  </td>
  <td class="sort-by-departure-date" data-value="1851-12-31">
   1851
  </td>
  <td class="sort-by-arrival-date" data-value="1851-12-31">
   1851
  </td>
 </tr>
 <tr>
  <td class="sort-by-name" data-value="Allred">
   <a href="/overlandtravel/companies/49/isaac-allred-freight-train">
    Isaac Allred Freig

In [31]:
for row in soup.tbody.contents[1::2]:
    print row.a.text.strip()
    print row.a.get('href')
    print row.find(class_='sort-by-number-of-pioneers').get('data-value')
    print row.find(class_='sort-by-departure-date').get('data-value')
    print row.find(class_='sort-by-arrival-date').get('data-value') 
    print

Horace M. Alexander Company (1847)
/overlandtravel/companies/402/horace-m-alexander-company
6
1847-07-24
1847-10-16

Levi Allred Company (1851)
/overlandtravel/companies/78747837894925591810/levi-allred-company
9
1851-12-31
1851-12-31

Isaac Allred Freight Train (1855)
/overlandtravel/companies/49/isaac-allred-freight-train
63
1855-07-28
1855-11-02

Fred C. Anderson Company (1868)
/overlandtravel/companies/50/fred-c-anderson-company
66
1868-01-01
1868-10-24

Milo Andrus Company (1855)
/overlandtravel/companies/52/milo-andrus-company
455
1855-08-04
1855-10-24

Milo Andrus Company (1861)
/overlandtravel/companies/53/milo-andrus-company
129
1861-07-07
1861-09-12

Milo Andrus Company (1850)
/overlandtravel/companies/51/milo-andrus-company
160
1850-06-03
1850-08-30

Elias Asper Freight Train (1861)
/overlandtravel/companies/55/elias-asper-freight-train
18
1861-06-19
1861-09-12

Atchison-Pilling Company (1853)
/overlandtravel/companies/490/atchison-pilling-company
13
1853-07-01
1853-09-13

W